In [1]:
import os
import pathlib
import sys

In [2]:
sys.path.append(str(pathlib.Path.cwd().parent.parent))

In [3]:
from src import feature_generator, numerical, categorical, dispatcher, cross_validation, utils, visualize

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression

In [5]:
wteams = pd.read_csv(f'{dispatcher.WDSTAGE1_TEAMS}')

In [11]:
wteams.TeamName.nunique()

365

In [13]:
wteams.loc[wteams.TeamID == 3467]

,TeamID,TeamName
364,3467,Merrimack


In [15]:
wseasons = pd.read_csv(f'{dispatcher.WDSTAGE1_SEASONS}')

In [16]:
wseasons.head()

,Season,DayZero,RegionW,RegionX,RegionY,RegionZ
0,1998,10/27/1997,East,Midwest,Mideast,West
1,1999,10/26/1998,East,Mideast,Midwest,West
2,2000,11/1/1999,East,Midwest,Mideast,West
3,2001,10/30/2000,East,Midwest,Mideast,West
4,2002,10/29/2001,East,West,Mideast,Midwest


In [17]:
wseasons.shape

(23, 6)

In [6]:
tourney_result = pd.read_csv(f'{dispatcher.WDSTAGE1_WNCAA_COMPACT_RESULTS}')

In [8]:
tourney_seed = pd.read_csv(f'{dispatcher.WDSTAGE1_WNCAA_SEEDS}')

In [9]:
tourney_result = tourney_result.drop(['DayNum', 'WScore', 'LScore', 'WLoc', 'NumOT'], axis=1)

In [11]:
tourney_result.head()

,Season,WTeamID,LTeamID
0,1998,3104,3422
1,1998,3112,3365
2,1998,3163,3193
3,1998,3198,3266
4,1998,3203,3208


In [12]:
tourney_seed.head()

,Season,Seed,TeamID
0,1998,W01,3330
1,1998,W02,3163
2,1998,W03,3112
3,1998,W04,3301
4,1998,W05,3272


In [13]:
tourney_result = pd.merge(tourney_result, tourney_seed, left_on=['Season', 'WTeamID'], right_on=['Season', 'TeamID'], how='left')
tourney_result.rename(columns={'Seed':'WSeed'}, inplace=True)
tourney_result = tourney_result.drop('TeamID', axis=1)
tourney_result = pd.merge(tourney_result, tourney_seed, left_on=['Season', 'LTeamID'], right_on=['Season', 'TeamID'], how='left')
tourney_result.rename(columns={'Seed':'LSeed'}, inplace=True)
tourney_result = tourney_result.drop('TeamID', axis=1)
tourney_result.head()

,Season,WTeamID,LTeamID,WSeed,LSeed
0,1998,3104,3422,X02,X15
1,1998,3112,3365,W03,W14
2,1998,3163,3193,W02,W15
3,1998,3198,3266,Y07,Y10
4,1998,3203,3208,W10,W07


In [14]:
def get_seed(x):
    return int(x[1:3])

In [15]:
from copy import deepcopy

In [16]:
tourney_result_tmp = deepcopy(tourney_result)

In [18]:
tourney_result['WSeed'] = tourney_result['WSeed'].map(lambda x: get_seed(x))
tourney_result['LSeed'] = tourney_result['LSeed'].map(lambda x: get_seed(x))
tourney_result.head()

,Season,WTeamID,LTeamID,WSeed,LSeed
0,1998,3104,3422,2,15
1,1998,3112,3365,3,14
2,1998,3163,3193,2,15
3,1998,3198,3266,7,10
4,1998,3203,3208,10,7


In [20]:
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion

In [25]:
a = Pipeline(
    [
        ('get_seed', FunctionTransformer(get_seed))
    ]
)

In [26]:
a.fit_transform(tourney_result_tmp[['WSeed']])

/home/huydg/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_function_transformer.py:98: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)


ValueError: could not convert string to float: 'X02'